# IpynbCompile.jl 交互式命令行

**✨执行其中所有单元格，可自动构建、测试并生成相应`.jl`源码**！

主要用途：为 ***IpynbCompile.jl*** 提供交互访问接口
- 可通过cmd命令行调用
    - 直接编译 语法：`compiler.ipynb.jl 文件名.ipynb`
- 可直接打开并进入「交互模式」
    - 直接键入路径，自动解析、编译并生成`.jl`文件

## 引入模块

In [1]:
# 编译后脚本中【直接引入】即可
LIB_IPYNB_PATH = "IpynbCompile.ipynb"
LIB_JL_PATH = "IpynbCompile.jl"
include(LIB_JL_PATH)

# %ignore-below
# ! 在Jupyter笔记本中执行时，将尝试编译"IpynbCompile.ipynb"并再度引入
IpynbCompile.compile_notebook(LIB_IPYNB_PATH, LIB_JL_PATH) # 编译
include(LIB_JL_PATH) # 再次引入

Main.IpynbCompile

In [2]:
# %ignore-cell # ! 尝试编译自身
SELF_FILE = "compiler.ipynb"
OUT_LIB_FILE = "compiler.jl"
IpynbCompile.compile_notebook(SELF_FILE, OUT_LIB_FILE)

3302

## 预置函数

In [3]:
try_compile_notebook(path, destination) = try
    printstyled("Compiling \"$path\" => \"$destination\"...\n", color=:white)
    local num_bytes = IpynbCompile.compile_notebook(path, destination)
    # 编译结果
    printstyled("[√] Compiling succeed with $num_bytes bytes!\n", color=:light_green)
catch e
    printstyled("[×] Compiling failed!\n", color=:light_red)
    @error e
    showerror(e)
end

try_compile_notebook (generic function with 1 method)

## 处理传入的命令行参数

处理方法：将其中所有参数视作**Jupyter笔记本路径**，直接【编译】成`.jl`文件

In [4]:
# ! 📝Julia对「命令行参数」的存储：【不包括】自身路径
function compile_with_ARGS(ARGS=ARGS)
    for path in ARGS
        try_compile_notebook(path)
    end
end

compile_with_ARGS (generic function with 2 methods)

## 交互模式

In [5]:
"""
交互模式
- 📌不断请求输入路径
"""
function interactive_mode()
    local path::AbstractString, out_path::AbstractString
    # 开始循环
    while true
        # 请求输入路径 | 空值⇒退出
        printstyled("IpynbCompile> path="; color=:light_cyan, bold=true)
        path = readline()
        # 空路径⇒退出
        if isempty(path)
            printstyled("Compiler exit!\n", color=:light_blue)
            return
        end
        # 先读取笔记本
        printstyled("Reading \"$path\"...\n", color=:white)
        local notebook = IpynbCompile.read_notebook(path)
        local lang = IpynbCompile.identify_lang(notebook)
        local default_out = "$path.$(IpynbCompile.get_extension(lang))"
        # 请求输出路径 | 默认为`输入路径.jl`
        printstyled("            > out_path(default \"$default_out\")="; color=:light_cyan, bold=true)
        out_path = readline()
        isempty(out_path) && (out_path = default_out) # 无⇒自动附加扩展名
        try_compile_notebook(path, out_path)
    end
end

interactive_mode

## 主程序

定义

In [6]:
"主程序"
function main()
    if isempty(ARGS)
        # 【无】附加参数时，进入交互模式
        interactive_mode()
    else
        # 有附加参数时⇒编译所有文件并自动退出
        compile_with_ARGS(ARGS)
    end
end

main

执行

In [7]:
#= %only-compiled # ! 编译后是直接执行的
main()
%only-compiled =#
# %ignore-below
#= # * 📝Jupyter笔记本中，ARGS传入的参数只有一个，并且内容类似「上下文路径」
{
    "key": "4a8bb47f-277d-47b5-b620-bcb1442ea48c",
    "signature_scheme": "hmac-sha256",
    "transport": "tcp",
    "ip": "127.0.0.1",
    "hb_port": 9010,
    "control_port": 9011,
    "shell_port": 9014,
    "stdin_port": 9015,
    "iopub_port": 9016,
    "kernel_name": "julia-1.9"
}
=#
# * Jupyter笔记本中：直接进入交互模式
interactive_mode()

IpynbCompile> path=Compiler exit!
